In [8]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from utils.data_processing_tick import convert_to_tick
from utils.holiday_utils import remove_friday
from utils.holiday_utils import extract_weekday
notebook_path = os.path.abspath(os.path.join(os.getcwd(), '../notebooks'))
from utils.histdata import import_pair
output_file_path = rf'{notebook_path}\data\processed'
model_output_filepath = rf'{notebook_path}\output\model'

**Get Data from histdata.com**

In [2]:
# Get intraday data from histdata.com
%debug
year = 2024
currency = "gbpusd"
import_pair(pair=currency, start=f"{year}01", end=f"{year}12", data_directory=r"data/raw")

ERROR:root:No traceback has been produced, nothing to debug.


Output()

Output()

Output()

Output()

**Create Renko Brick Data**

In [6]:
# Convet_to_tick function : create renko data from intraday historical data
%debug

year = "2024"
ccy = "gbpusd"
brick_size = 0.001 # 0.001 for EURUSD/EURGBP/GBPUSD, 0.1 for USDJPY
use_first_price_as_base=False # True: use first price as base, False: use previous price as base

csv_filepath = rf'{notebook_path}\data\raw\ASCII\M1\{ccy.lower()}\{year}'
csv_filename = f'DAT_ASCII_{ccy.upper()}_M1_{year}.csv'

if (use_first_price_as_base):
    output_filename = f'{year}_{ccy.lower()}_tick_first_price_as_base.csv'
else:
    output_filename = f'{year}_{ccy.lower()}_tick_prev_price_as_base.csv'

convert_to_tick(csv_filepath, csv_filename, output_file_path, output_filename, brick_size, use_first_price_as_base) 

ERROR:root:No traceback has been produced, nothing to debug.


Tick-like data has been saved to tick_like_data.csv
   tick_number            datetime    price  actual_openprice  ticks_moved  \
0            1 2024-01-01 17:46:00  1.27284           1.27287            1   
1            2 2024-01-01 21:38:00  1.27184           1.27171            1   
2            3 2024-01-02 00:43:00  1.27284           1.27290            1   
3            4 2024-01-02 00:53:00  1.27384           1.27395            1   
4            5 2024-01-02 03:07:00  1.27484           1.27506            1   

   directions  fx_return day_of_week  
0           1        NaN      Monday  
1          -1  -0.091133      Monday  
2           1   0.093575     Tuesday  
3           1   0.082489     Tuesday  
4           1   0.087131     Tuesday  


,tick_number,datetime,price,actual_openprice,ticks_moved,directions,fx_return,day_of_week
0,1,2024-01-01 17:46:00,1.27284,1.27287,1,1,NaN,Monday
1,2,2024-01-01 21:38:00,1.27184,1.27171,1,-1,-0.091133,Monday
2,3,2024-01-02 00:43:00,1.27284,1.27290,1,1,0.093575,Tuesday
3,4,2024-01-02 00:53:00,1.27384,1.27395,1,1,0.082489,Tuesday
4,5,2024-01-02 03:07:00,1.27484,1.27506,1,1,0.087131,Tuesday
...,...,...,...,...,...,...,...,...
5135,5136,2024-12-31 12:54:00,1.25184,1.25190,1,1,0.088744,Tuesday
5136,5137,2024-12-31 13:31:00,1.25084,1.25068,1,-1,-0.097452,Tuesday
5137,5138,2024-12-31 14:15:00,1.25184,1.25190,1,1,0.097547,Tuesday
5138,5139,2024-12-31 14:48:00,1.25084,1.25083,1,-1,-0.085470,Tuesday


In [17]:
import os
import re
import glob
import pandas as pd

filename_pattern = re.compile(
    r"(\d{4})_(\w+)_(\w+(?:_\w+)*)_(random_forest_results|backtesting)\.csv"
)

summary_data = {}

data_dir = rf'{model_output_filepath}\2023'
csv_files = glob.glob(os.path.join(data_dir, "*.csv"))

print("Files found:", csv_files)
for filepath in csv_files:
    filename = os.path.basename(filepath)
    print("Checking:", filename)

    match = filename_pattern.match(filename)
    if not match:
        print("  --> Regex did NOT match. Skipping.")
        continue

    year, ccy, session, file_type = match.groups()
    print(f"  --> Regex matched: year={year}, ccy={ccy}, session={session}, file_type={file_type}")
    key = (year, ccy, session)

    if key not in summary_data:
        summary_data[key] = {
            "year": year,
            "ccy": ccy,
            "session": session,
            "accuracy": None,
            "backtest_start": None,
            "backtest_end": None,
            "closing_balance": None
        }
    
    df = pd.read_csv(filepath)
    print("  --> DataFrame columns:", df.columns.tolist())

    if file_type == "random_forest_results":
        # Make sure "Accuracy" actually exists in df.columns
        summary_data[key]["accuracy"] = df["Accuracy"].iloc[0]
    else:  # backtesting
        # Make sure "datetime" and "balance" exist in df.columns
        summary_data[key]["backtest_start"] = df["datetime"].iloc[0]
        summary_data[key]["backtest_end"]   = df["datetime"].iloc[-1]
        summary_data[key]["closing_balance"] = df["balance"].iloc[-1]

# After the loop, see what we collected
print("\nSummary so far:")
for k, v in summary_data.items():
    print(k, "->", v)

output_df = pd.DataFrame(summary_data.values())
output_file = os.path.join(data_dir, "final_summary.csv")
output_df.to_csv(output_file, index=False)
print("Final summary saved to:", output_file)


Files found: ['c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_asian_backtesting.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_asian_morning_backtesting.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_asian_morning_random_forest_results.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_asian_random_forest_results.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_london_afternoon_backtesting.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_london_afternoon_random_forest_results.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2023_eurgbp_london_backtesting.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\2023\\2